# Initialisation

In [33]:
#!pip install --upgrade google-api-python-client

## Imports

In [34]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoModelForCausalLM , AutoTokenizer, pipeline, RobertaTokenizer, RobertaForMaskedLM, AlbertTokenizer, AlbertModel, AlbertForMaskedLM
import logging
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from afinn import Afinn
from tqdm import tqdm
import unidecode
from time import sleep
from googleapiclient import discovery
import json
logging.basicConfig(level=logging.INFO)# OPTIONAL

## MAC Settings

In [35]:
print(f"PyTorch version: {torch.__version__}")
# Set the device      
device = "mps" if torch.backends.mps.is_available() else torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

PyTorch version: 1.13.1
Using device: mps


## Definitions

In [ ]:
IDENTITIES = 'identities'
ORIENTATION = 'orientation'
OTHER = 'other'
PRONOUNS = 'pronouns'
MASKBERT_ = '\[MASK\]'
MASKBERT= '[MASK]'
MASKROBERT = '<mask>'
TARGET = '<target>'
NOM = '<nom>'
POSDEP = '<pos_dep>'
ACC = '<acc>'
BE = '<be>'
QUEER = 'queer'
NONQUEER = 'non-queer'

#TEMPLATES
TEMPLATE_NOZZA = '../src/templates/template_nozza.csv'
TEMPLATE_NOZZA_COMPLETE = '../src/templates/template_nozza_complete.csv'
TEMPLATE_TOXIC2 = '../src/templates/template_toxic2.csv'
TEMPLATE_TOXIC2_COMPLETE = '../src/templates/template_toxic2_complete.csv'
TEMPLATE_TOXIC1 = '../src/templates/template_toxic1.csv'
TEMPLATE_TOXIC1_COMPLETE = '../src/templates/template_toxic1_complete.csv'
TEMPLATE_TOXIC1_CHUNK = '../src/templates/toxic1/template_toxic1'
TEMPLATE_TOXIC2_CHUNK = '../src/templates/toxic2/template_toxic2'

#IDENTITIES CSV
IDENTITIES_CSV = '../src/queer_identities/identities.csv'
PRONOUNS_CSV = '../src/queer_identities/pronouns.csv'

#MODELS
BERT_BASE = 'bert-base-uncased'
BERT_LARGE = 'bert-large-uncased'
ROBERTA_BASE = 'roberta-base'
ROBERTA_LARGE = 'roberta-large'
GPT2 = 'gpt2'

# Queer Bench - Evaluator

In [39]:
def QueerEvaluator(model_name, template_path):
    def __init__(self):
        self.model_name = model_name
        self.template_path = template_path
        
    
